In [1]:
#==============================================================================
# 05-model-second-stage.R
# Purpose: running second stage of the model (estimation of mass-level ideal
# points) with a parallelized metropolis algorithm
# Author: Pablo Barbera
#==============================================================================

library(parallel)
library(dplyr)
library(Matrix)

# install.packages('devtools', repos="http://cran.us.r-project.org", dependencies = TRUE)
# library(devtools)
# library(tidyverse)
# library(arm)

# install.packages("remotes")
# library(remotes)

# remotes::install_github("pablobarbera/twitter_ideology/pkg/tweetscores", dependencies = TRUE)
library(tweetscores)

source('utils.R')
source('auxiliary_functions/functions.R')

samplesfile <- 'data/output/samples-tcc-filter-br.rdata'
matrixfile <- 'data/output/adj-matrix-tcc-filter-br.rdata'

# resultfile <- 'data/output/results-elites-tcc-filter-br.rdata'
resultfile <- 'data/output/results-elites-tcc-filter-br-new.rdata'

# loading results of first stage
load(samplesfile)
alpha.i <- samples$alpha
gamma.i <- samples$gamma
phi.i <- samples$phi
mu_beta.i <- samples$mu_beta
sigma_beta.i <- samples$sigma_beta

# loading data matrix
library(Matrix)
load(matrixfile)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: R2WinBUGS

Loading required package: coda

Loading required package: boot

##
## tweetscores: tools for the analysis of Twitter data

## Pablo Barbera (USC)

## www.tweetscores.com
##



In [2]:
# # Warnings

# ## log posterior density (Maximum Likelihood)
# lpd.ml <- function(pars, alpha, gamma, phi, mu_beta, sigma_beta, y){
#   beta <- pars[1]
#   theta <- pars[2]

#   value <- alpha + beta - gamma * (theta - phi)^2
#   lk <- sum(y * plogis(value, log.p=TRUE) + (1-y) * plogis(-value, log.p=TRUE)) +
#     dnorm(theta, 0, 1, log=TRUE) + dnorm(beta, mean=mu_beta, sd=sigma_beta, log=TRUE)
#   return(-lk)
# }



# ml.logit <- function(y,
#                      alpha.i,
#                      gamma.i,
#                      phi.i,
#                      mu_beta.i,
#                      sigma_beta.i,
#                      beta.init=log(sum(y)),
#                      theta.init=rnorm(1, 0, 1),
#                      iters=5000, chains=2, n.warmup=1000, thin=20, verbose=TRUE)
# {
#   # Preparando vetores para armazenar amostras
#   sims <- length(seq(n.warmup+1, iters, by=thin))
#   pars.samples <- array(NA, dim=c(sims, chains, 2), dimnames=list(NULL, NULL, c("beta", "theta")))

#   # Iterações para cada cadeia
#   for (chain in 1:chains) {
#     # Pontos de partida
#     pars.cur <- c(beta.init, theta.init)
#     for (iter in 1:iters) {
#       # Valores atuais dos parâmetros
#       alpha <- alpha.i[iter, ]
#       gamma <- gamma.i[iter]
#       phi <- phi.i[iter, ]
#       mu_beta <- mu_beta.i[iter]
#       sigma_beta <- sigma_beta.i[iter]

#       # Maximização da função de verossimilhança
#       res <- optim(par=pars.cur, fn=lpd.ml, alpha=alpha,
#                    gamma=gamma, phi=phi, mu_beta=mu_beta, sigma_beta=sigma_beta,
#                    y=y * 1, hessian=TRUE)
#       sds <- sqrt(diag(solve(res$hessian)))

#       # Amostragem de distribuição dos parâmetros
#       pars.samples[, chain, "beta"] <- rnorm(sims, res$par[1], sds[1])
#       pars.samples[, chain, "theta"] <- rnorm(sims, res$par[2], sds[2])

#       # Atualizar os parâmetros atuais para o próximo loop
#       pars.cur <- res$par

#       if (verbose && iter %% 100 == 0) {
#         cat("Iteração:", iter, "Cadeia:", chain, "\n")
#       }
#     }
#   }

#   # Estatísticas resumidas
#   results <- round(R2WinBUGS::monitor(pars.samples), 2)
#   if (verbose) {
#     message("")
#     print(results)
#   }

#   return(list(samples=pars.samples, Rhat=results[,"Rhat"], n.eff=results[,"n.eff"]))
# }





# ## versão paralelizada do algoritmo de estimação
# estimation <- function(first, last=first+4999){
#     pars <- first:last
#     beta.samples <- array(NA, dim=c(length(pars), 200), 
#         dimnames=list(paste("beta[", first:last, "]", sep=""), 
#             paste("Iteration ", 1:200, sep="")))
#     theta.samples <- array(NA, dim=c(length(pars), 200), 
#         dimnames=list(paste("theta[", first:last, "]", sep=""), 
#             paste("Iteration ", 1:200, sep="")))
#     theta.results <- data.frame(theta=NA, 
#         id=dimnames(y)[[1]][1:length(pars)], rhat=NA, n.eff=NA,
#         stringsAsFactors=F) 

#     for (i in 1:length(pars)){
#         fit <- ml.logit(y[pars[i],], alpha.i, gamma.i, phi.i, mu_beta.i, sigma_beta.i,
#             beta.init=log(sum(y[pars[i],])), theta.init=rnorm(1,0,1), iters=3000, 
#             chains=2, n.warmup=1000, thin=20, verbose=FALSE)
#         beta.samples[i,] <- c(fit$samples[,,"beta"])
#         theta.samples[i,] <- c(fit$samples[,,"theta"])
#         theta.results$theta[i] <- mean(theta.samples[i,])
#         theta.results$rhat[i] <- fit$Rhat[2]
#         theta.results$n.eff[i] <- fit$n.eff[2]
#         cat(pars[i], "\n")
#     }
#     return(list(beta.samples=beta.samples, theta.samples=theta.samples, 
#         theta.results=theta.results))
# }


In [3]:
## log posterior density
lpd <- function(alpha, beta, gamma, theta, phi, mu_beta, sigma_beta, y){
    require(arm, quiet=TRUE)
    value <- alpha + beta - gamma * (theta - phi)^2
    sum(log(invlogit(value)^y * (1-invlogit(value))^(1-y)))  + 
         dnorm(theta, 0, 1, log=TRUE) + dnorm(beta, mean=mu_beta, sd=sigma_beta, log=TRUE)
}

## substituindo metropolis.logit por ml.logit na função estimation
estimation <- function(first, last=first+4999){
    pars <- first:last
    beta.samples <- array(NA, dim=c(length(pars), 200), 
        dimnames=list(paste("beta[", first:last, "]", sep=""), 
            paste("Iteration ", 1:200, sep="")))
    theta.samples <- array(NA, dim=c(length(pars), 200), 
        dimnames=list(paste("theta[", first:last, "]", sep=""), 
            paste("Iteration ", 1:200, sep="")))
    theta.results <- data.frame(theta=NA, 
        id=dimnames(y)[[1]][1:length(pars)], rhat=NA, n.eff=NA,
        stringsAsFactors=F) 

    for (i in 1:length(pars)){
        fit <- ml.logit(y[pars[i],], alpha=colMeans(alpha.i), gamma=mean(gamma.i), phi=colMeans(phi.i),
                        beta.init=log(sum(y[pars[i],])), theta.init=rnorm(1,0,1),
                        mu_beta=mean(mu_beta.i), sigma_beta=mean(sigma_beta.i), 
                        iters=3000, chains=2, n.warmup=1000, thin=20, verbose=FALSE)
        beta.samples[i,] <- c(fit$samples[,,"beta"])
        theta.samples[i,] <- c(fit$samples[,,"theta"])
        theta.results$theta[i] <- mean(theta.samples[i,])
        theta.results$rhat[i] <- fit$Rhat[2]
        theta.results$n.eff[i] <- fit$n.eff[2]
        cat(pars[i], "\n")
    }
    return(list(beta.samples=beta.samples, theta.samples=theta.samples, 
        theta.results=theta.results))
}

In [4]:
# first batch of results
n1 <- 1
n2 <- 50000

## preparando dados
load(matrixfile)
y <- y[n1:(n1+49999),]
results1 <- mclapply(seq(n1,n2,5000), estimation, mc.cores=12)
save(results1, file="temp/results1.Rdata")

In [ ]:
stop

In [ ]:
# repeat code above for results 2 to 6
n1 <- 50001
n2 <- 100000
load(matrixfile)
y <- y[n1:(n1+49999),]
results2 <- mclapply(seq(n1,n2,5000), estimation, mc.cores=1)
save(results2, file="temp/results2.Rdata")

n1 <- 100001
n2 <- 150000
load(matrixfile)
y <- y[n1:(n1+49999),]
results3 <- mclapply(seq(n1,n2,5000), estimation, mc.cores=1)
save(results3, file="temp/results3.Rdata")

n1 <- 150001
n2 <- 200000
load(matrixfile)
y <- y[n1:(n1+49999),]
results4 <- mclapply(seq(n1,n2,5000), estimation, mc.cores=1)
save(results4, file="temp/results4.Rdata")

n1 <- 200001
n2 <- 250000
load(matrixfile)
y <- y[n1:(n1+49999),]
results5 <- mclapply(seq(n1,n2,5000), estimation, mc.cores=12)
save(results5, file="temp/results5.Rdata")

n1 <- 250001
n2 <- 300000
load(matrixfile)
y <- y[n1:(n1+49999),]
results6 <- mclapply(seq(n1,n2,5000), estimation, mc.cores=12)
save(results6, file="temp/results6.Rdata")

## last batch of results
load(matrixfile)
y <- y[300001:dim(y)[1],]
results7 <- estimation(first=1, last=dim(y)[1])
save(results7, file="temp/results7.Rdata")


ERROR: Error in intI(i, n = x@Dim[1], dn[[1]], give.dn = FALSE): index larger than maximal 50000


In [ ]:
#####################################################################
##### PUTTING IT ALL TOGETHER
#####################################################################
load(matrixfile)

load("temp/results1.Rdata")
load("temp/results2.Rdata")
load("temp/results3.Rdata")
load("temp/results4.Rdata")
load("temp/results5.Rdata")
load("temp/results6.Rdata")
load("temp/results7.Rdata")


beta.samples <- list()
j <- 1

for (i in 1:length(results1)){
    beta.samples[[j]] <- results1[i][[1]][['beta.samples']]
    j <- j + 1
}
for (i in 1:length(results2)){
    beta.samples[[j]] <- results2[i][[1]][['beta.samples']]
    j <- j + 1
}
for (i in 1:length(results3)){
    beta.samples[[j]] <- results3[i][[1]][['beta.samples']]
    j <- j + 1
}
for (i in 1:length(results4)){
    beta.samples[[j]] <- results4[i][[1]][['beta.samples']]
    j <- j + 1
}
for (i in 1:length(results5)){
    beta.samples[[j]] <- results5[i][[1]][['beta.samples']]
    j <- j + 1
}
for (i in 1:length(results6)){
    beta.samples[[j]] <- results6[i][[1]][['beta.samples']]
    j <- j + 1
}
beta.samples[[j]] <- results7[['beta.samples']]

beta.samples <- t(do.call(rbind, beta.samples))


theta.samples <- list()
j <- 1

for (i in 1:length(results1)){
    theta.samples[[j]] <- results1[i][[1]][['theta.samples']]
    j <- j + 1
}
for (i in 1:length(results2)){
    theta.samples[[j]] <- results2[i][[1]][['theta.samples']]
    j <- j + 1
}
for (i in 1:length(results3)){
    theta.samples[[j]] <- results3[i][[1]][['theta.samples']]
    j <- j + 1
}
for (i in 1:length(results4)){
    theta.samples[[j]] <- results4[i][[1]][['theta.samples']]
    j <- j + 1
}
for (i in 1:length(results5)){
    theta.samples[[j]] <- results5[i][[1]][['theta.samples']]
    j <- j + 1
}
for (i in 1:length(results6)){
    theta.samples[[j]] <- results6[i][[1]][['theta.samples']]
    j <- j + 1
}

theta.samples[[j]] <- results7[['theta.samples']]

theta.samples <- t(do.call(rbind, theta.samples))

rm("results1")
rm("results2")
rm("results3")
rm("results4")
rm("results5")
rm("results6")
rm("results7")

samples <- list(
    alpha = samples$alpha,
    beta = beta.samples,
    gamma = samples$gamma,
    theta = theta.samples,
    phi = samples$phi)



Warning message in readChar(con, 5L, useBytes = TRUE):
“cannot open compressed file 'temp/results1.Rdata', probable reason 'No such file or directory'”


ERROR: Error in readChar(con, 5L, useBytes = TRUE): cannot open the connection


In [ ]:
### RESULTS ####

load("temp/results1.Rdata")
load("temp/results2.Rdata")
load("temp/results3.Rdata")
load("temp/results4.Rdata")
load("temp/results5.Rdata")
load("temp/results6.Rdata")
load("temp/results7.Rdata")

theta.results <- list()
j <- 1
for (i in 1:length(results1)){
    theta.results[[j]] <- results1[i][[1]][['theta.results']]
    j <- j + 1
}
for (i in 1:length(results2)){
    theta.results[[j]] <- results2[i][[1]][['theta.results']]
    j <- j + 1
}
for (i in 1:length(results3)){
    theta.results[[j]] <- results3[i][[1]][['theta.results']]
    j <- j + 1
}
for (i in 1:length(results4)){
    theta.results[[j]] <- results4[i][[1]][['theta.results']]
    j <- j + 1
}
for (i in 1:length(results5)){
    theta.results[[j]] <- results5[i][[1]][['theta.results']]
    j <- j + 1
}
for (i in 1:length(results6)){
    theta.results[[j]] <- results6[i][[1]][['theta.results']]
    j <- j + 1
}
theta.results[[j]] <- results7[['theta.results']]

theta.results <- do.call(rbind, theta.results)

In [ ]:
#==============================================================================
## THIS IS THE KEY PART
## 'RESULTS' is a list that contains all sets of parameter estimates:
## theta = ideology for ordinary users
## theta.sd = standard deviation of ideology for ordinary users
## phi, phi.sd = same for politicians
## m.names = names of politicians (twitter handles)
## n.names = IDs of users
## rhat.theta = Rhat for users (measures convergence of MCMC chains)


results <- list(
    alpha = apply(samples$alpha, 2, mean),
    beta = apply(samples$beta, 2, mean),
    gamma = mean(samples$gamma),
    theta = apply(samples$theta, 2, mean),
    theta.sd = apply(samples$theta, 2, sd),
    phi = apply(samples$phi, 2, mean),
    phi.sd = apply(samples$phi, 2, sd),
    m.names = dimnames(y)[[2]],
    n.names = dimnames(y)[[1]],
    rhat.theta = theta.results$rhat
    )

save(results, file=resultsfile)